### INTRODUCTION

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def describe_more(df,normalize_ind=False, weight_column=None, skip_columns=[], dropna=True) :
    var = [] ; l = [] ; t = []; unq =[]; min_l = []; max_l = [];
    assert isinstance(skip_columns, list), "Argument skip_columns should be list"
    if weight_column is not None:
        if weight_column not in list(df.columns):
            raise AssertionError('weight_column is not a valid column name in the input DataFrame')
      
    for x in df:
        if x in skip_columns:
            pass
        else:
            var.append( x )
            uniq_counts = len(pd.value_counts(df[x],dropna=dropna))
            uniq_counts = len(pd.value_counts(df[x], dropna=dropna)[pd.value_counts(df[x],dropna=dropna)>0])
            l.append(uniq_counts)
            t.append( df[ x ].dtypes )
            min_l.append(df[x].apply(str).str.len().min())
            max_l.append(df[x].apply(str).str.len().max())
            if weight_column is not None and x not in skip_columns:
                df2 = df.groupby(x).agg({weight_column: 'sum'}).sort_values(weight_column, ascending=False)
                df2['authtrans_vts_cnt']=((df2[weight_column])/df2[weight_column].sum()).round(2)
                unq.append(df2.head(n=100).to_dict()[weight_column])
            else:
                df_cat_d = df[x].value_counts(normalize=normalize_ind,dropna=dropna).round(decimals=2)
                df_cat_d = df_cat_d[df_cat_d>0]
                #unq.append(df[x].value_counts().iloc[0:100].to_dict())
                unq.append(df_cat_d.iloc[0:100].to_dict())
            
    levels = pd.DataFrame( { 'A_Variable' : var , 'Levels' : l , 'Datatype' : t ,
                             'Min Length' : min_l, 'Max Length': max_l, 'Level_Values' : unq} )
    return levels

In [ ]:
df=pd.read_csv("/Users/onkar/Downloads/SBA_archive/SBAnational.csv")

In [ ]:
df.head()

In [ ]:
df.shape

## 1. DATA CLEANING

##### According to the dataset documentation provided :

There are a number of variables that consistently emerge as indicators of risk that could explain the variation of loan default rates. Seven variables, along with some exploratory analysis, are discussed below including Location (State), Industry, Gross Disbursement, New vs Established Business, Loans Backed by Real Estate, Economic Recession, and SBA's Guaranteed Portion of Approved Loan.
 

In [ ]:
df=df.drop_duplicates(keep='first')   # delete duplicates, if any

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(subset=['MIS_Status','RevLineCr','LowDoc','Bank','BankState',
                  'Name','City','State','NewExist','DisbursementDate'],inplace=True)

# dropped all but ChgOffDate null values as we do not really need that column in particular

In [ ]:
df['MIS_Status'].value_counts()   # we want to see how many loans have defaulted, indicated by status as 'CHGOFF'

In [ ]:
df=df.drop(axis=1,columns=['ChgOffDate'])   # dropped ChgOffDate column

In [ ]:
describe_more(df)


##### # We see here that some records of DisbursementGross, BalanceGross, ChgOffPrinGr, GrAppv & SBA_Appv are being read as objects (strings) instead of numbers (integers)


##### > Some things we note or want to do are: 

     1. drop records that dont have Y or N values of LowDoc
     2. some ApprovalFY records are also not integers
     3. drop NewExist records that have value 0
     4. convert ApprovalDate and DisbursementDate columns to datetime values
 

In [ ]:
cols_mess=["DisbursementGross", "BalanceGross", "ChgOffPrinGr", "GrAppv", "SBA_Appv"]
df[cols_mess]=df[cols_mess].astype(str).replace("[']","",regex=True)   # drop single quotation marks
df[cols_mess]=df[cols_mess].astype(str).replace("[,]","",regex=True)   # drop commas
df[cols_mess]=df[cols_mess].astype(str).replace("[$]","",regex=True)   # drop $ sign

df[cols_mess]=df[cols_mess].astype(float)   # convert the cols_mess columns to float data type

df[cols_mess].info()

In [ ]:
df["ApprovalFY"].unique()   # note that some years are within quotation marks and one is followed by an A

In [ ]:
df["ApprovalFY"]=df["ApprovalFY"].astype(str).replace("'","",regex=True)
df["ApprovalFY"]=df["ApprovalFY"].astype(str).replace("A","",regex=True)

df["ApprovalFY"]=df["ApprovalFY"].astype(int)   # convert ApprovalFY dtype from object to int

df["ApprovalFY"].dtype

In [ ]:
df = df[(df['LowDoc']=='Y') | (df['LowDoc']=='N')]   # dropped all but Y/N records in LowDoc
df['LowDoc'].isna().sum()

In [ ]:
df = df[(df['NewExist']==1) | (df['NewExist']==2)]   # dropped unspecified values (0) in NewExist
df["NewExist"]=df["NewExist"].astype(int)

print(df["NewExist"].isna().sum())
print(df["NewExist"].dtype)

##### # Some of the fields that are considered flags already but aren't necessarily in a useable format right now. 
These include the NewExist, RevLineCr, LowDoc, and MIS_Status fields; some of which need fixing. 
##### 

In [ ]:
df['MIS_Status'].where(~(df["MIS_Status"]=='P I F'), other=0, inplace=True)    # replace PIF as 0
df['MIS_Status'].where(~(df["MIS_Status"]=='CHGOFF'), other=1, inplace=True)   # replace CHGOFF as 1

df["MIS_Status"]=df["MIS_Status"].astype(int)
df["MIS_Status"].dtype

In [ ]:
df['LowDoc'].where(~(df["LowDoc"]=='N'), other=0, inplace=True)   # replace N as 0
df['LowDoc'].where(~(df["LowDoc"]=='Y'), other=1, inplace=True)   # replace Y as 1

df["LowDoc"]=df["LowDoc"].astype(int)
df["LowDoc"].dtype

##### > We can now work to clean RevLineCr which has a large records (especially of 0) having unneccessary values

In [ ]:
df["RevLineCr"].value_counts()

In [ ]:
df = df[(df['RevLineCr']=='Y') | (df['RevLineCr']=='N') | (df['RevLineCr']=='0')]
df["RevLineCr"].value_counts()   # dropped all garbage value of RevLineCr and left 0 as unknown factor

In [ ]:
df['RevLineCr'].where(~(df["RevLineCr"]=='N'), other=0, inplace=True)   # replace N as 0
df['RevLineCr'].where(~(df["RevLineCr"]=='Y'), other=1, inplace=True)   # replace Y as 1
df['RevLineCr'].where(~(df["RevLineCr"]==0), other=-1, inplace=True)    # replace 0 as -1

df["RevLineCr"]=df["RevLineCr"].astype(int)
df["RevLineCr"].dtype

##### > Convert ApprovalDate and DisbursementDate columns to datetime values

In [ ]:
from datetime import date

df[['ApprovalDate','DisbursementDate']]=df[['ApprovalDate','DisbursementDate']].apply(
                                            lambda x: pd.to_datetime(x,errors = 'coerce'))

##### > Check if all missing values have been dealt with

In [ ]:
df.isna().sum()

##### > Check whether all columns we manipulated have the right datatype

In [ ]:
df.info()

## 2. Feature Engineering

##### > We have franchise codes which are unneccessary, we need only know whether a business is a franchise or not

In [ ]:
df.loc[(df['FranchiseCode'] <= 1),'Franchise'] = 0
df.loc[(df['FranchiseCode'] > 1),'Franchise'] = 1

df["Franchise"]=df["Franchise"].astype(int)      # Convert datatype of Franchise to int
df.drop(columns="FranchiseCode", inplace=True)   # Drop FranchiseCode since its of no particular use to us

##### > Create a new column named "Industry" with the industry the NAICS code represents

In [ ]:
df['Industry'] = df['NAICS'].astype('str').apply(lambda x: x[:2])   # Select only first 2 numbers of NAICS code

df['Industry'] = df['Industry'].map({
    '11': 'Ag/For/Fish/Hunt',
    '21': 'Min/Quar/Oil_Gas_ext',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale_trade',
    '44': 'Retail_trade',
    '45': 'Retail_trade',
    '48': 'Trans/Ware',
    '49': 'Trans/Ware',
    '51': 'Information',
    '52': 'Finance/Insurance',
    '53': 'RE/Rental/Lease',
    '54': 'Prof/Science/Tech',
    '55': 'Mgmt_comp',
    '56': 'Admin_sup/Waste_Mgmt_Rem',
    '61': 'Educational',
    '62': 'Healthcare/Social_assist',
    '71': 'Arts/Entertain/Rec',
    '72': 'Accom/Food_serv',
    '81': 'Other_no_pub',
    '92': 'Public_Admin'
})   # Map the approprate industry to each record based on the first two digits of the NAICS code

df.dropna(subset=['Industry'], inplace=True)   # Remove records where Industry is NaN (NAICS code was 0)
df.drop(columns="NAICS", inplace=True)         # Drop NAICS since its of no particular use to us

##### > Feature showing the guaranteed amount as percentage of the gross loan amount

In [ ]:
df['SBA_ApvPct']=round(df['SBA_Appv']/df['GrAppv'],3)

##### > Feature for loans backed by Real Estate (loans with a term of at least 20 years)

In [ ]:
df['RealEstate']=np.where(df['Term']>=240,1,0)

##### > Feature for loans active during the Great Recession (2007-2009)

In [ ]:
df['DisbursementFY']=df['DisbursementDate'].map(lambda x: x.year)   # find year of disbursement

df['GreatRecession']=np.where(((2007 <= df['DisbursementFY']) & (df['DisbursementFY'] <= 2009)) | 
                        ((df['DisbursementFY'] < 2007) & (df['DisbursementFY'] + (df['Term']/12) >= 2007)), 1, 0)

##### > Select only records with a disbursement year through 2010

In [ ]:
df=df[df['DisbursementFY']<=2010]

##### # According to dataset document: 
An emphasis is placed on the default rates of loans with a disbursement date through 2010. We chose this time period for two reasons. We want to account for variation due to the Great Recession (December 2007 to June 2009); so loans disbursed before, during, and after this time frame are needed. Secondly, we restrict the time frame to loans by  excluding those disbursed after 2010 due to the fact the term of a loan is frequently 5 or more years.

##### > We will now drop columns that are not of use to us

In [ ]:
df.drop(columns=['LoanNr_ChkDgt','Name','City','Zip','Bank','BankState', 'NoEmp', 'CreateJob', 'RetainedJob',
                 'ChgOffPrinGr','ApprovalDate','DisbursementDate'], inplace=True)

##### # The columns I dropped and the reason why is stated below -

LoanNr_ChkDgt, Name, City, Zip, Bank, BankState, NoEmp, CreateJob, RetainedJob - provide no value to the actual analysis

ApprovalDate -  unneccessary as we already have ApprovalFY

DisbursementDate - replaced by DisbursementFY

ChgOffPrinGr - amount not of use to us, we already have MIS_Status

We dont drop State as we want to check the effect of demograpphy on the loan default

## 3. Outlier Detection & Removal

##### > Let's check the Gross Disbursement column

In [ ]:
import plotly.express as px

for i in df:
    fig = px.histogram(df, x=i, color_discrete_sequence=px.colors.qualitative.Antique, text_auto=True)
    fig.show()

##### # Looking at these histograms, here are some attributes we see that need attention -

1. Term should not be more than 300 months (25 years)
2. DisbursementGross should not be more than 2.0M
3. GrAppv & SBA_Appv should not be more than 0.5M

In [ ]:
df = df[df['Term'] <= 300]
df = df[df['DisbursementGross'] <= 2000000]
df = df[df['GrAppv'] <= 500000]
df = df[df['SBA_Appv'] <= 500000]

print("Max Term:",df['Term'].max())
print("Max Disbursement Gross:",df['DisbursementGross'].max())
print("Max Gross Approved:",df['GrAppv'].max())
print("Max SBA Approved :",df['SBA_Appv'].max())

## 4. Exploratory Data Analysis

In [ ]:
df.describe()

##### # INSIGHTS:

 1. Mean of SBA Approved Percentage is about 60% of total loan amount needed
 2. Most loans were disbursed between 1997-2008 (approval FY)
 3. Loans backed by Real Estate are about 10% of total loans
 4. An average of 75% loans were active during the Great Recession (2007-2009)
 5. Only about 5% of loans were given to Franchises, while 27.78% loans were given to new businesses
 6. More than 90% of loans were given to businesses in an Urban area
 7. Average term of loan was around 8 years with a std. dev. of 5 years
 8. More than 10% of loans sanctioned were LowDoc program (1 page loan doc for amount <150k)
 9. Average loan Gross Disbursement was around 120k with under 75% of them being around 160k

In [ ]:
plt.figure(figsize=(15,7))
sns.histplot(df.GrAppv, color="green", kde=False)
plt.ylabel('Density')
plt.title('Distribution of Approved ammount')
plt.show()

##### # Observation - This plot shows the right skewedness of GrAppv, so we fix the skewness using a log function

In [ ]:
df['GrAppv']=np.log(df['GrAppv'])

## 5. Data Visualization

In [ ]:
sns.barplot(x='MIS_Status', y='Term', data=df)
plt.title('Average Term vs MIS_Status')

In [ ]:
sns.barplot(x='MIS_Status', y='DisbursementGross', data=df)
plt.title('Average DisbursementGross vs MIS_Status')

In [ ]:
sns.barplot(x='MIS_Status', y='GrAppv', data=df)
plt.title('Average GrAppv vs MIS_Status')

In [ ]:
sns.barplot(x='MIS_Status', y='SBA_Appv', data=df)
plt.title('Average SBA_Appv vs MIS_Status')

In [ ]:
# Total/Average disbursed loan amount by industry
# Create a groupby object on Industry for use in visualization
industry_group = df.groupby(['Industry'])

# Data frames based on groupby by Industry looking at aggregate and average values
df_industrySum = industry_group.sum().sort_values('DisbursementGross', ascending=False)
df_industryAve = industry_group.mean().sort_values('DisbursementGross', ascending=False)

# Establish figure for placing bar charts side-by-side
fig = plt.figure(figsize=(25, 10))

# Add subplots to figure to build 1x2 grid and specify position of each subplot
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

# ______ Bar chart 1 = Gross SBA Loan Disbursement by Industry ______

ax1.bar(df_industrySum.index, df_industrySum['DisbursementGross'] / 1000000000)
ax1.set_xticklabels(df_industrySum.index, rotation=30, horizontalalignment='right', fontsize=10)

ax1.set_title('Gross SBA Loan Disbursement by Industry from 1984-2010', fontsize=15)
ax1.set_xlabel('Industry')
ax1.set_ylabel('Gross Loan Disbursement (Billions)')

# ______ Bar chart 2 = Average SBA Loan Disbursement by Industry ______

ax2.bar(df_industryAve.index, df_industryAve['DisbursementGross'])
ax2.set_xticklabels(df_industryAve.index, rotation=30, horizontalalignment='right', fontsize=10)

ax2.set_title('Average SBA Loan Disbursement by Industry from 1984-2010', fontsize=15)
ax2.set_xlabel('Industry')
ax2.set_ylabel('Average Loan Disbursement')

plt.show()

#### > Loans categorized by Industry and State

In [ ]:
# Paid in full and defaulted loans

fig3 = plt.figure(figsize=(15, 10))

ax1a = plt.subplot(2, 1, 1)
ax2a = plt.subplot(2, 1, 2)

# Function for creating stacked bar charts grouped by desired column
# df = original data frame, col = x-axis grouping, stack_col = column to show stacked values
# Essentially acts as a stacked histogram when stack_col is a flag variable

def stacked_setup(df, col, axes, stack_col='MIS_Status'):
    data = df.groupby([col, stack_col])[col].count().unstack(stack_col)
    data.fillna(0)

    axes.bar(data.index, data[1], label='Default')
    axes.bar(data.index, data[0], bottom=data[1], label='Paid in full')


# ______ Number of Paid in full and defaulted loans by INDUSTRY ______

stacked_setup(df=df, col='Industry', axes=ax1a)
ax1a.set_xticklabels(df.groupby(['Industry', 'MIS_Status'])['Industry'].count().unstack('MIS_Status').index,
                     rotation=35, horizontalalignment='right', fontsize=10)

ax1a.set_title('Number of PIF/Defaulted Loans by Industry from 1984-2010', fontsize=15)
ax1a.set_xlabel('Industry')
ax1a.set_ylabel('Number of PIF/Defaulted Loans')
ax1a.legend()

# ______ Number of Paid in full and defaulted loans by STATE ______

stacked_setup(df=df, col='State', axes=ax2a)

ax2a.set_title('Number of PIF/Defaulted Loans by State from 1984-2010', fontsize=15)
ax2a.set_xlabel('State')
ax2a.set_ylabel('Number of PIF/Defaulted Loans')
ax2a.legend()

plt.tight_layout()
plt.show()

##### > Default percentage by Industry

In [ ]:
def_ind = df.groupby(['Industry', 'MIS_Status'])['Industry'].count().unstack('MIS_Status')
def_ind['Def_Percent'] = def_ind[1]/(def_ind[1] + def_ind[0])
def_ind

##### > Check Default percentage by State

In [ ]:
def_state = df.groupby(['State', 'MIS_Status'])['State'].count().unstack('MIS_Status')
def_state['Def_Percent'] = def_state[1]/(def_state[1] + def_state[0])
def_state

##### > Loans paid in full and Defaulted loans by DisbursementFY

In [ ]:
fig4, ax4 = plt.subplots(figsize=(15, 5))

stack_data = df.groupby(['DisbursementFY', 'MIS_Status'])['DisbursementFY'].count().unstack('MIS_Status')
x = stack_data.index
y = [stack_data[1], stack_data[0]]

ax4.stackplot(x, y, labels=['Default', 'Paid in full'])
ax4.set_title('Number of PIF/Defaulted Loans by State from 1984-2010', fontsize=15)
ax4.set_xlabel('Disbursement Year')
ax4.set_ylabel('Number of PIF/Defaulted Loans')
ax4.legend(loc='upper left')

plt.show()

# We use a stacked area chart here since it's time series data

##### # It can be clearly seen that most loans have defaulted in the time-period leading up to the Great Reccession 2008 Also note how the number of loans have icreased until the Great Reccesion and then sharply decreased

##### > Loans backed by Real Estate v/s Loans during the Great Recession

In [ ]:
# _____ Paid in full and defaulted loans backed by Real Estate ______

fig5 = plt.figure(figsize=(20, 10))

ax1b = fig5.add_subplot(1, 2, 1)
ax2b = fig5.add_subplot(1, 2, 2)

stacked_setup(df=df, col='RealEstate', axes=ax1b)
ax1b.set_xticks(df.groupby(['RealEstate', 'MIS_Status'])['RealEstate'].count().unstack('MIS_Status').index)
ax1b.set_xticklabels(labels=['No', 'Yes'])

ax1b.set_title('Number of PIF/Defaulted Loans backed by Real Estate from 1984-2010', fontsize=15)
ax1b.set_xlabel('Loan Backed by Real Estate')
ax1b.set_ylabel('Number of Loans')
ax1b.legend()

# ______ Paid in full and defaulted loans active during the Great Recession ______

stacked_setup(df=df, col='GreatRecession', axes=ax2b)
ax2b.set_xticks(df.groupby(['GreatRecession', 'MIS_Status'])['GreatRecession'].count().unstack('MIS_Status').index)
ax2b.set_xticklabels(labels=['No', 'Yes'])

ax2b.set_title('Number of PIF/Defaulted Loans Active during the Great Recession from 1984-2010', fontsize=15)
ax2b.set_xlabel('Loan Active during Great Recession')
ax2b.set_ylabel('Number of Loans')
ax2b.legend()

plt.show()

##### # Surprisingly, the volume of loans backed by real estate was much less than those not backed by real estate however the default rate is also much less for loans backed by real estate.

##### > Default percentage for loans backed by Real Estate

In [ ]:
def_re = df.groupby(['RealEstate', 'MIS_Status'])['RealEstate'].count().unstack('MIS_Status')
def_re['Def_Percent'] = def_re[1]/(def_re[1] + def_re[0])
def_re

##### > Default percentage for loans active during the Great Recession

In [ ]:
def_gr = df.groupby(['GreatRecession', 'MIS_Status'])['GreatRecession'].count().unstack('MIS_Status')
def_gr['Def_Percent'] = def_gr[1]/(def_gr[1] + def_gr[0])
def_gr

## 6. Model Building

##### > Dummy encoding

In [ ]:
df=pd.get_dummies(df)
df.head()

##### > Split data into train and test sets + label target value

In [ ]:
y = df.MIS_Status
X = df.drop(['MIS_Status'], axis=1)

# Scale the feature values prior to modeling
scale = StandardScaler()
X_scaled = scale.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.25)

#### > LOGISTIC REGRESSION

In [ ]:
log_reg = LogisticRegression(random_state=0)   # Initialize model

log_reg.fit(X_train, y_train)
y_logpred = log_reg.predict(X_val)   # Train model and make predictions

print(classification_report(y_val, y_logpred, digits=3))   # Print results

##### # Observation -

We can see here that with the Logistic Regression model, we have a surprising accuracy of 85%, and the F1-score of 60% for defaulted loans doesn't seem promising at all. The precision suggests that the model is correct 70% of the time when the loan defaults, and the recall suggests that the model identifies only a 50% of defaulted loans correctly. That means that every one of two loans that defaulted were incorrectly classified as loans that would be paid in full, which is horribly bad business for any loan providing entity.

#### > DECISION TREES

In [ ]:
dtc = DecisionTreeClassifier(random_state=0)
dtc.fit(X_train, y_train)

pred = dtc.predict(X_val)

print(classification_report(y_val, pred))
print(confusion_matrix(y_val, pred))

##### # note how our scores have increased vastly as compared to Logistic Regression

#### > XGBOOST

In [ ]:
xgboost = XGBClassifier(random_state=0)

xgboost.fit(X_train, y_train)
y_xgbpred = xgboost.predict(X_val)

print(classification_report(y_val, y_xgbpred, digits=3))

##### # scores have increased slightly more than Decision Trees, we can still tune hyperparameters if we want to

#### > List the importance of each feature

In [ ]:
for name, importance in sorted(zip(X.columns, xgboost.feature_importances_)):
    print(name, "=", round(importance*100,3), "%")

##### # Observations -

Top 5 features : 
    Term (17%)
    --> ApprovalFY (6%)
    --> SBA_ApvPct (3.4%)
    --> State_CA (3.3%)
    --> State_FL (2.8%)


Top 3 industries : 
    Healthcare (2.2%)
    --> Rental/Lease (1.1%) 
    --> Retail_Trade (1.1%)

RealEstate has 0 effect on loan default, understandably since people taking such a risk have a pretty good plan

##### > Let's see if reducing the number of features used to the most important ones would have a positive impact on the model performance, since the current model has a high level of dimensionality.

In [ ]:
from sklearn.pipeline import Pipeline   # Build pipeline for feature selection and modeling 
from sklearn.feature_selection import SelectKBest   # SelectKBest defaults to top 10 features

xgb_featimp = XGBClassifier(random_state=0)
pipe = Pipeline(steps=[ ('feature_selection', SelectKBest()), ('model', xgb_featimp) ])

pipe.fit(X_train, y_train)
y_featimppred = pipe.predict(X_val)

print(classification_report(y_val, y_featimppred, digits=3))

##### # Reducing the number of features, and thereby dimensionality of the data, didn't affect the results too much.

##### > List the importance of each feature for the lower dimensionality dataframe

In [ ]:
for name, importance in sorted(zip(X.columns, xgb_featimp.feature_importances_)):
    print(name, "=", round(importance*100,3), "%")

##### # Observation -

Top 5 features are now Term (50%), ApprovalFY (16%), BalanceGross (10%), SBA_Appv (5.5%), NewExist (4.5%)

Note how ApprovalFY and Term are the only initial features on this new Top 5 list

### CONCLUSION

##### According to the analysis, the factor that contributed the most to whether or not a loan defaulted is the loan's Term length.